<a href="https://colab.research.google.com/github/ykitaguchi77/Kaggle/blob/main/H-M.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

Mounted at /content/drive


#**Open transaction file**

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Deep_learning/Kaggle_H&M/transactions_train.csv')

In [ ]:
#Pickle形式で一旦保存（Loadが早くなる）
df.to_pickle('/content/drive/MyDrive/Deep_learning/Kaggle_H&M/transactions_train.pickle') # pickle ファイルに変換

In [2]:
train = pd.read_pickle('/content/drive/MyDrive/Deep_learning/Kaggle_H&M/transactions_train.pickle') # pickleファイルの読み込み
train

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2
...,...,...,...,...,...
31788319,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,929511001,0.059305,2
31788320,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,891322004,0.042356,2
31788321,2020-09-22,fff380805474b287b05cb2a7507b9a013482f7dd0bce0e...,918325001,0.043203,1
31788322,2020-09-22,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,833459002,0.006763,1


#**重複を削除したカスタマーリストを作成**

In [10]:
customer_id_list = train.iloc[:, 1]
customer_id_list = customer_id_list[~customer_id_list.duplicated()].reset_index(drop=True)
customer_id_list

0          000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...
1          00007d2de826758b65a93dd24ce629ed66842531df6699...
2          00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...
3          0008968c0d451dbc5a9968da03196fe20051965edde741...
4          000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...
                                 ...                        
1362276    fe99a0069d6b3c64c2707d0ce53b9311540917471d82df...
1362277    fecc5f77b5f7ee4570efde9ab05ec94d0de2bf80efb4f6...
1362278    fece2f68864c311a0b5208e2eb735b3dcde7e41461d327...
1362279    fee56cc5315dafb35a4490ccc6f711092cae913550c832...
1362280    ff5b8a8b26bf93a66290e9bd1b73393ac6a58968a78519...
Name: customer_id, Length: 1362281, dtype: object

In [8]:
print(len(customer_id_list)) #1362281人のデータ

1362281


In [13]:
a = train[train.iloc[:,1] == customer_id_list[3]]
len(a)

61

#**重複を削除した商品リストを作成**

In [ ]:
article_list = train.iloc[:, 2]
article_list = article_list[~article_list.duplicated()].reset_index(drop=True)
article_list #商品が104546個もある！！

0         663713001
1         541518023
2         505221004
3         685687003
4         685687004
            ...    
104542    913290001
104543    860135008
104544    539060028
104545    810737015
104546    533261032
Name: article_id, Length: 104547, dtype: int64

In [ ]:
len(article_list)

104547

#**類似商品のクラスタリング**
https://qiita.com/kotai2003/items/55c4f525ff216ca16800

In [3]:
!pip install pycaret
import pycaret
from pycaret.datasets import get_data


     |████████████████████████████████| 301 kB 5.2 MB/s 
     |████████████████████████████████| 15.6 MB 46.4 MB/s 
     |████████████████████████████████| 6.8 MB 47.8 MB/s 
     |████████████████████████████████| 167 kB 41.4 MB/s 
     |████████████████████████████████| 114 kB 63.7 MB/s 
     |████████████████████████████████| 274 kB 58.6 MB/s 
     |████████████████████████████████| 56 kB 2.9 MB/s 
     |████████████████████████████████| 1.7 MB 36.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 86 kB 4.6 MB/s 
     |████████████████████████████████| 2.0 MB 18.6 MB/s 
     |████████████████████████████████| 1.3 MB 14.4 MB/s 
     |████████████████████████████████| 261 kB 61.4 MB/s 
     |████████████████████████████████| 303 kB 46.1 MB/s 
     |████████████████████████████████| 675 kB 53.6 MB/s 
     |█████████████████

In [ ]:
import pandas as pd
import pandas_profiling
df_article = pd.read_csv("/content/drive/MyDrive/Deep_learning/Kaggle_H&M/articles.csv")

#パイロット用に少量のデータをピックアップする
df_article = df_article.iloc[:500, :]
df_article

In [ ]:
#商品リストの概要
#df_article = df_article[["product_type_name","product_group_name","graphical_appearance_name", "colour_group_name","perceived_colour_value_name", "perceived_colour_master_name", "department_name", "index_name", "garment_group_name"]]
df_article.profile_report()

In [116]:
print(df_article["index_group_name"].value_counts())
"""
ここでメンズ、ウィメンズ、子供用の区分けをする
Sports --> ほとんどは男性用。暫定的に男性服に分類する
"""

Ladieswear       245
Divided           98
Baby/Children     92
Menswear          44
Sport             21
Name: index_group_name, dtype: int64


'\nメンズ、ウィメンズの区分けぐらいはしておいた方が良さそう\n'

In [67]:
print(df_article.columns)
print(df_article['index_name'].value_counts())

"""
子供服を年齢層別に分類可能
"""

Index(['article_id', 'product_code', 'prod_name', 'product_type_no',
       'product_type_name', 'product_group_name', 'graphical_appearance_no',
       'graphical_appearance_name', 'colour_group_code', 'colour_group_name',
       'perceived_colour_value_id', 'perceived_colour_value_name',
       'perceived_colour_master_id', 'perceived_colour_master_name',
       'department_no', 'department_name', 'index_code', 'index_name',
       'index_group_no', 'index_group_name', 'section_no', 'section_name',
       'garment_group_no', 'garment_group_name', 'detail_desc'],
      dtype='object')
Lingeries/Tights          108
Divided                    98
Ladieswear                 80
Baby Sizes 50-98           61
Ladies Accessories         57
Menswear                   44
Children Sizes 134-170     23
Sport                      21
Children Sizes 92-140       8
Name: index_name, dtype: int64


In [136]:
print(df_article["product_type_name"].value_counts())

"""
男女別に、これらの項目に分ける。
項目 
Leggings/Tights, Underwear Tights, Underwear bottom
Sweater, Top, Hoodie, Vest top, Cardigan
Bodysuit
Trousers, Skirt
Pyjama jumpsuit/playsuit, Pyjama set
Socks
T-shirt
Hair clip, Hair string, Hair/alice band
Swimsuit, Bikini top, Swimwear bottom
Gloves
Bra
Hat/beanie, Cap/peaked
Sunglasses
Belt
Blazer, Sneakers, Boots
Sleep Bag
Shirt
Shorts
Bag
Unknown
Umbrella
Earring
"""

Leggings/Tights             50
Trousers                    45
Pyjama jumpsuit/playsuit    39
Socks                       37
Hoodie                      36
Underwear Tights            34
Dress                       24
T-shirt                     23
Sweater                     23
Hair clip                   16
Vest top                    15
Swimsuit                    13
Top                         13
Gloves                      12
Hair string                 12
Bra                         11
Bodysuit                     9
Pyjama set                   9
Hat/beanie                   9
Sunglasses                   8
Belt                         8
Blazer                       7
Sleep Bag                    5
Kids Underwear top           5
Shirt                        5
Skirt                        5
Cap/peaked                   4
Sneakers                     4
Hair/alice band              4
Shorts                       3
Bag                          2
Underwear bottom             2
Bikini t

'\n男女別に、これらの項目に分ける。\n項目 \nLeggings/Tights, Underwear Tights, Underwear bottom\nSweater, Top, Hoodie, Vest top, Cardigan\nBodysuit\nTrousers, Skirt\nTrousers\nPyajama jumpsuit/playsuit, Pyajama set\nSocks\nT-shirt\nHair clip, Hair string, Hair/alice band\nSwimsuit, Bikini top, Swimwear bottom\nGloves\nBra\nHat/beanie, Cap/peaked\nSunglasses\nBelt\nBlazer, Sneakers, Boots\nSleep Bag\nShirt\nShorts\nBag\nUnderwear bottom\nUnknown\nUmbrella\nEarring\n'

In [144]:
"""
方針：
index_group_nameにより、メンズ、ウィメンズ、子供用商品を分別する
メンズ、ウィメンズ別に、product type nameで分別
"""

def concat_df(df, column, name_list): #column:"section_name", name_list=["Womens Everyday Basics", "Womens Lingerie"] 
    df_list = [0] * len(name_list)
    for i in range(len(name_list)):
        df_list[i] = df.loc[df[column] == name_list[i]]
        #print(df_list[i])
    df_concat = pd.concat(df_list)
    return df_concat

def extract_df(df, column, name):
    df = df.loc[df[column] == name]
    return df



In [170]:
pd.set_option('display.max_rows', 500)


df_baby = extract_df(df_article, "index_name", "Baby Sizes 50-98")
df_children_small = extract_df(df_article, "index_name", "Children Sizes 92-140")
df_children_large = extract_df(df_article, "index_name", "Children Sizes 134-170")

df_womens = concat_df(df_article, "index_name", ["Ladieswear", "Divided"])
df_mens = concat_df(df_article, "index_name", ["Menswear", "Sport"])

df_women_tights = concat_df(df_womens, "product_type_name", ["Leggings/Tights", "Underwear bottom", "Underwear Tights"])
df_women_tops = concat_df(df_womens, "product_type_name", ["Sweater", "Top", "Hoodie", "Vest top", "Cardigan"])
df_women_bodysuits = concat_df(df_womens, "product_type_name", ["Bodysuit"])
df_women_pyjamas = concat_df(df_womens, "product_type_name", ["Pyjama jumpsuit/playsuit", "Pyjama set"])
df_women_socks = concat_df(df_womens, "product_type_name", ["Socks"])
df_women_tshirt = concat_df(df_womens, "product_type_name", ["T-shirt"])
df_women_hair = concat_df(df_womens, "product_type_name", ["Hair clip", "Hair string", "Hair/alice band"])
df_women_swimsuits = concat_df(df_womens, "product_type_name", ["Swimsuit", "Bikini top", "Swimwear bottom"])
df_women_gloves = concat_df(df_womens, "product_type_name", ["Gloves"])
df_women_bra = concat_df(df_womens, "product_type_name", ["Belt"])
df_women_shoes = concat_df(df_womens, "product_type_name", ["Blazer", "Sneakers", "Boots"])
df_women_sleepbag = concat_df(df_womens, "product_type_name", ["Sleep Bag"])
df_women_shirts = concat_df(df_womens, "product_type_name", ["Shirt"])
df_women_shorts = concat_df(df_womens, "product_type_name", ["Shorts"])
df_women_bags = concat_df(df_womens, "product_type_name", ["Bag"])
df_women_unknown = concat_df(df_womens, "product_type_name", ["Unknown"])
df_women_umbrellas = concat_df(df_womens, "product_type_name", ["Umbrella"])
df_women_earrings = concat_df(df_womens, "product_type_name", ["Earring"])

df_men_tights = concat_df(df_mens, "product_type_name", ["Leggings/Tights", "Underwear bottom", "Underwear Tights"])
df_men_tops = concat_df(df_mens, "product_type_name", ["Sweater", "Top", "Hoodie", "Vest top", "Cardigan"])
df_men_bodysuits = concat_df(df_mens, "product_type_name", ["Bodysuit"])
df_men_pyjamas = concat_df(df_mens, "product_type_name", ["Pyjama jumpsuit/playsuit", "Pyjama set"])
df_men_socks = concat_df(df_mens, "product_type_name", ["Socks"])
df_men_tshirt = concat_df(df_mens, "product_type_name", ["T-shirt"])
df_men_hair = concat_df(df_mens, "product_type_name", ["Hair clip", "Hair string", "Hair/alice band"])
df_men_swimsuits = concat_df(df_mens, "product_type_name", ["Swimsuit", "Bikini top", "Swimwear bottom"])
df_men_gloves = concat_df(df_mens, "product_type_name", ["Gloves"])
df_men_bra = concat_df(df_mens, "product_type_name", ["Belt"])
df_men_shoes = concat_df(df_mens, "product_type_name", ["Blazer", "Sneakers", "Boots"])
df_men_sleepbag = concat_df(df_mens, "product_type_name", ["Sleep Bag"])
df_men_shirts = concat_df(df_mens, "product_type_name", ["Shirt"])
df_men_shorts = concat_df(df_mens, "product_type_name", ["Shorts"])
df_men_bags = concat_df(df_mens, "product_type_name", ["Bag"])
df_men_unknown = concat_df(df_mens, "product_type_name", ["Unknown"])
df_men_umbrellas = concat_df(df_mens, "product_type_name", ["Umbrella"])
df_men_earrings = concat_df(df_mens, "product_type_name", ["Earring"])


In [ ]:
df_article["item_type"] = 0 #最終行に追加
df_article.info()

In [194]:
df_article.loc[44,"item_type"]  = "baby"

In [203]:
for i in range(len(df_article)):
    if df_article.loc[i,"index_name"] == "Baby Sizes 50-98":
        df_article.loc[i,"item_type"]  = "baby"
    elif df_article.loc[i,"index_name"] == "Children Sizes 92-140":
        df_article.loc[i,"item_type"]  = "children_small"
    elif df_article.loc[i,"index_name"] == "Children Sizes 134-170":
        df_article.loc[i,"item_type"]  = "children_large"
    elif df_article.loc[i,"index_name"] == "Ladieswear" or "Divided":
        if df_article.loc[i,"product_type_name"] == "Leggings/Tights" or "Underwear bottom" or "Underwear Tights":
            df_article.loc[i,"item_type"]  = "tights_womens"
        if df_article.loc[i,"product_type_name"] == "Sweater" or "Top" or "Hoodie" or "Vest top" or "Cardigan":
            df_article.loc[i,"item_type"]  = "tops_womens"
        if df_article.loc[i,"product_type_name"] == "Bodysuit":
            df_article.loc[i,"item_type"]  = "bodysuits_womens"
        if df_article.loc[i,"product_type_name"] == "Pyjama jumpsuit/playsuit" or "Pyjama set":
            df_article.loc[i,"item_type"]  = "pyjyamas_womens"
        if df_article.loc[i,"product_type_name"] == "Leggings/Tights" or "Underwear bottom" or "Underwear Tights":
            df_article.loc[i,"item_type"]  = "tights_womens"
        if df_article.loc[i,"product_type_name"] == "Sweater" or "Top" or "Hoodie" or "Vest top" or "Cardigan":
            df_article.loc[i,"item_type"]  = "tops_womens"
        if df_article.loc[i,"product_type_name"] == "Bodysuit":
            df_article.loc[i,"item_type"]  = "bodysuits_womens"
        if df_article.loc[i,"product_type_name"] == "Sweater" or "Top" or "Hoodie" or "Vest top" or "Cardigan":
            df_article.loc[i,"item_type"]  = "tops_womens"

KeyError: ignored

In [202]:
df_article["item_type"]

0       tights_womens
1       tights_womens
2       tights_womens
3       tights_womens
4       tights_womens
5       tights_womens
6       tights_womens
7       tights_womens
8       tights_womens
9       tights_womens
10      tights_womens
11               baby
12               baby
13      tights_womens
14      tights_womens
15      tights_womens
16      tights_womens
17      tights_womens
18      tights_womens
19      tights_womens
20      tights_womens
21      tights_womens
22      tights_womens
23      tights_womens
24      tights_womens
25      tights_womens
26      tights_womens
27      tights_womens
28      tights_womens
29      tights_womens
30      tights_womens
31      tights_womens
32      tights_womens
33      tights_womens
34      tights_womens
35      tights_womens
36      tights_womens
37               baby
38               baby
39      tights_womens
40      tights_womens
41      tights_womens
42      tights_womens
43      tights_womens
44      tights_womens
45      ti

In [57]:
"""
def set_feature(dst_list, loc, new_class):
    if dst_list[loc] == 0:
        dst_list[loc] = new_class
    else:
        pass
    return dst_list


def make_article_class_list(df, class_list, dst_list): #列全体に文字列が含まれるかをチェック
    for classes in class_list:
        for i in range(len(df)):
            for j in range(len(df.columns)):
                if df.iloc[i,:].str.contains(classes)[j] is True:
                    dst_list = set_feature(dst_list, i, classes)
    return dst_list

def make_article_class_list_single(df, column, class_list, dst_list): #特定のセルに文字列が含まれるかをチェック
    article_class_list = [0] * product_num
    for classes in class_list:
        for i in range(len(df)):
            if classes in df.iloc[i,17]:
                dst_list = set_feature(dst_list, i, classes)
    return dst_list
"""


In [ ]:
class_list = ["Baby Sizes 50-98", "Children Sizes 92-140", "Children Sizes 134-170"]

product_num = len(df_article)
dst_list = [0] * product_num
dst_list = make_article_class_list(df_article, class_list, dst_list)

In [59]:
#リストをイニシャライズ
product_num = len(df_article)
dst_list = [0] * product_num

class_list = ["Baby Sizes 50-98", "Children Sizes 92-140", "Children Sizes 134-170"]
dst_list = make_article_class_list_single(df_article, 17, class_list, dst_list)
dst_list

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 'Baby Sizes 50-98',
 'Baby Sizes 50-98',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 'Baby Sizes 50-98',
 'Baby Sizes 50-98',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 'Baby Sizes 50-98',
 'Baby Sizes 50-98',
 'Baby Sizes 50-98',
 0,
 0,
 0,
 'Baby Sizes 50-98',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 'Children Sizes 92-140',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 'Children Sizes 134-170',
 'Children Sizes 134-170',
 0,
 0,
 'Baby Sizes 50-98',
 'Baby Sizes 50-98',
 'Baby Sizes 50-98',
 'Baby Sizes 50-98',
 'Baby Sizes 50-98',
 'Baby Sizes 50-98',
 'Baby Sizes 50-98',
 'Baby Sizes 50-98',
 'Baby Sizes 50-98',
 'Baby Sizes 50-98',
 'Baby Sizes 50-98',
 'Baby Sizes 50-98',
 'Baby Sizes 50-98',
 'Baby Sizes 50-98',
 0]

In [55]:
df_article.iloc[1,17]

'Ladieswear'

In [56]:
dst_list

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 'Baby Sizes 50-98',
 'Baby Sizes 50-98',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 'Baby Sizes 50-98',
 'Baby Sizes 50-98',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 'Baby Sizes 50-98',
 'Baby Sizes 50-98',
 'Baby Sizes 50-98',
 0,
 0,
 0,
 'Baby Sizes 50-98',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 'Children Sizes 92-140',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 'Children Sizes 134-170',
 'Children Sizes 134-170',
 0,
 0,
 'Baby Sizes 50-98',
 'Baby Sizes 50-98',
 'Baby Sizes 50-98',
 'Baby Sizes 50-98',
 'Baby Sizes 50-98',
 'Baby Sizes 50-98',
 'Baby Sizes 50-98',
 'Baby Sizes 50-98',
 'Baby Sizes 50-98',
 'Baby Sizes 50-98',
 'Baby Sizes 50-98',
 'Baby Sizes 50-98',
 'Baby Sizes 50-98',
 'Baby Sizes 50-98',
 0]

In [ ]:
data = article.sample(frac=0.95, random_state=786).reset_index(drop=True)
data_unseen = article.drop(data.index).reset_index(drop=True)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

Data for Modeling: (475, 9)
Unseen Data For Predictions: (25, 9)


In [ ]:
from pycaret.clustering import *
data_clust = setup(data, normalize = True, session_id = 123)

In [ ]:
kmeans = create_model('kmeans',num_clusters = 7 )
print(kmeans)

,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness
0,0.159,35.8255,2.3639,0,0,0


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=7, n_init=10, n_jobs=-1, precompute_distances='deprecated',
       random_state=123, tol=0.0001, verbose=0)


In [ ]:
kmean_results = assign_model(kmeans)
kmean_results.head()

,product_type_name,product_group_name,graphical_appearance_name,colour_group_name,perceived_colour_value_name,perceived_colour_master_name,department_name,index_name,garment_group_name,Cluster
0,Underwear Tights,Socks & Tights,Solid,Black,Dark,Black,Tights basic,Lingeries/Tights,Socks and Tights,Cluster 1
1,Trousers,Garment Lower body,Denim,Light Blue,Dusty Light,Blue,Woven bottoms,Ladieswear,Trousers,Cluster 6
2,Top,Garment Upper body,Solid,Grey,Dusty Light,Grey,Kids Boy Jersey Basic,Children Sizes 92-140,Jersey Basic,Cluster 5
3,Hair/alice band,Accessories,Solid,Light Orange,Dusty Light,Orange,Hair Accessories,Ladies Accessories,Accessories,Cluster 3
4,Pyjama jumpsuit/playsuit,Nightwear,All over pattern,Light Red,Medium,Red,Baby Nightwear,Baby Sizes 50-98,"Under-, Nightwear",Cluster 4


In [ ]:
pd.set_option('display.max_rows', 500)
kmean_results

In [ ]:
#PCA plot
plot_model(kmeans)

In [ ]:
#Elbow plot (suggested cluster number)
plot_model(kmeans, plot = 'elbow')

In [ ]:
#Distribution plot
plot_model(kmeans, plot = 'distribution', feature = 'class')

In [ ]:
#Predictions
unseen_predictions = predict_model(kmeans, data=data_unseen)
unseen_predictions.head()

In [ ]:
"""
種類 → product type name, product group name：　下着（上下）、アンダー、シャツ、ジャケット、アクセサリー
男性物、女性物、子供、赤ちゃん

どのように分類するか？
Mens/Ladies/Kids Boy/Kids Girl/Babies
Product type name/Product group name/Garment group name
Colour  perceived color master, perceived color name 
Childrenはサイズが分かれている

このあたりで大まかに分類し、購入の傾向をつかむ
→後で細分化する
"""